In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import os
import scipy.io as sio
from scipy.signal import resample

In [2]:
picks = [2,4,7,9,11,14,16]
not_picks = [i for i in range(25) if i not in picks]
not_picks

[0, 1, 3, 5, 6, 8, 10, 12, 13, 15, 17, 18, 19, 20, 21, 22, 23, 24]

In [3]:
def read_dataset(dataset,num):
    path = "dataset"+str(dataset)+"/"
    if dataset == 1:
        path += "A"
    elif dataset == 2:
        path += "S"
    elif dataset == 4:
        path += "B"
    e = []
    t = []
    for i in range(1,num+1):
        e.append(np.asarray(sio.loadmat(path+str(i).zfill(2)+"E.mat")['data']))
        t.append(np.asarray(sio.loadmat(path+str(i).zfill(2)+"T.mat")['data']))
    return t, e

In [4]:
t, e = read_dataset(1,9)
t4, e4 = read_dataset(4,9)

In [5]:
t[0][0][5][0]

array([(array([[10.83984375, 11.18164062,  9.91210938, ..., 13.18359375,
         8.7890625 ,  0.        ],
       [16.65039062, 18.79882812, 16.16210938, ..., 14.16015625,
        11.71875   ,  8.7890625 ],
       [10.30273438, 14.30664062, 11.03515625, ...,  5.37109375,
         3.90625   ,  3.90625   ],
       ...,
       [10.15625   , 12.5       ,  9.52148438, ..., 17.578125  ,
        23.92578125,  2.9296875 ],
       [11.03515625, 14.2578125 , 11.08398438, ...,  7.8125    ,
        16.6015625 ,  9.765625  ],
       [ 6.640625  ,  9.71679688,  6.10351562, ...,  3.90625   ,
        11.71875   ,  8.7890625 ]]), array([[  251],
       [ 2254],
       [ 4172],
       [ 6124],
       [ 8132],
       [10243],
       [12160],
       [14210],
       [16141],
       [18139],
       [20045],
       [21940],
       [23912],
       [25856],
       [27823],
       [29943],
       [31951],
       [34017],
       [36053],
       [38119],
       [40189],
       [42269],
       [44181],
       [46

In [6]:
for i,x in enumerate(t):
    print(i,len(x[0]))

0 9
1 9
2 9
3 7
4 9
5 9
6 9
7 9
8 9


In [7]:
len(t[3][0])

7

In [8]:
t[3][0][5][0][0][0].shape

(96735, 25)

In [166]:
for i in range(len(t4)):
    for j in range(len(t4[i][0])):
        print(i,j,t4[i][0][j][0][0][0].shape)
    print('*'*25)

0 0 (604803, 6)
0 1 (552452, 6)
0 2 (469011, 6)
*************************
1 0 (604803, 6)
1 1 (604803, 6)
1 2 (464155, 6)
*************************
2 0 (604803, 6)
2 1 (604803, 6)
2 2 (464675, 6)
*************************
3 0 (604803, 6)
3 1 (696266, 6)
3 2 (468559, 6)
*************************
4 0 (604803, 6)
4 1 (696266, 6)
4 2 (461875, 6)
*************************
5 0 (604803, 6)
5 1 (604803, 6)
5 2 (470391, 6)
*************************
6 0 (604803, 6)
6 1 (604803, 6)
6 2 (466727, 6)
*************************
7 0 (787729, 6)
7 1 (604803, 6)
7 2 (474835, 6)
*************************
8 0 (604803, 6)
8 1 (604803, 6)
8 2 (460731, 6)
*************************


In [186]:
def prep_data(data):
    dataset = []
    for i in range(len(data[0])):
        dataset.append(data[0][i])
    return np.array(dataset)


In [190]:
raw_data1t = [prep_data(tt)[1:] if i == 3 else prep_d1(tt)[3:] for i,tt in enumerate(t)]
raw_data1e = [prep_data(ee)[3:] for ee in e]
raw_data4t = [prep_data(tt) for tt in t4]
raw_data4e = [prep_data(ee) for ee in e4]

In [188]:
aa = raw_data1t[5][0]

In [189]:
aa[0][0][1][0][0]

251

In [14]:
def extract_data(raw_data):
    data, label = [], []
    for i,x1 in enumerate(raw_data):
        d, l = [], []
        for j in range(len(x1[0][0][1])):
            if x1[0][0][5][j][0] == 0:
                if j == len(x1[0][0][5]) - 1:
                    d.append(x1[0][0][0][x1[0][0][1][-1][0]:])
                else:
                    d.append(x1[0][0][0][x1[0][0][1][j][0]:x1[0][0][1][j+1][0]])
                l.append(x1[0][0][2][j][0])
        data.append(np.array(d))
        label.append(np.array(l))
    return np.array([data,label])

In [15]:
data_1t = np.array([extract_data(data) for data in raw_data1t])
data_1e = np.array([extract_data(data) for data in raw_data1e])

In [16]:
data_1t[0][0][4][0].shape

(2003, 25)

In [19]:
def down_data(raw_x,raw_y):
    new_x = []
    new_y = []
    sum_val, cnt, max_shape = 0, 0, 0
    
    for i, x1 in enumerate(raw_x):
        tmp_x1, tmp_y1 = [], []
        
        for j, x2 in enumerate(x1):
            tmp_x2, tmp_y2 = [], []
            
            for k, x3 in enumerate(x2):
                if raw_y[i][j][k] in [1,2]:
                    tmp_y2.append(raw_y[i][j][k])
                    tmp_x2.append(np.delete(x3,not_picks,axis=1))
                    sum_val += x3.shape[0]
                    cnt += 1
            if len(tmp_x2) > max_shape:
                max_shape = len(tmp_x2)
            
            tmp_x1.append(tmp_x2)
            tmp_y1.append(tmp_y2)
        new_x.append(tmp_x1)
        new_y.append(tmp_y1)
    
    down_size = int((sum_val / cnt) * 160 / 250) 
    
    x = np.zeros((len(raw_x),len(raw_x[0]),max_shape,down_size,7),dtype=np.float64)
    y = np.zeros((len(raw_x),len(raw_x[0]),max_shape),dtype=np.int32)
    
    for i in range(len(new_x)):
        for j in range(len(new_x[i])):
            y[i,j,:len(new_x[i][j])] = new_y[i][j]
            for k in range(len(new_x[i][j])):
                x[i,j,k] = resample(new_x[i][j][k],down_size,axis=0)
    
    return x, y

In [216]:
def down_data2(raw_x,raw_y,dataset):
    new_x = []
    new_y = []
    sum_val, cnt = 0, 0
    
    for i, x1 in enumerate(raw_x):
        tmp_x1, tmp_y1 = [], []
        
        for j, x2 in enumerate(x1):
            tmp_x2 = []
            
            for k, x3 in enumerate(x2):
                if raw_y[i][j][k] in [1,2]:
                    tmp_y1.append(raw_y[i][j][k])
                    if int(dataset) == 1:
                        tmp_x2.append(np.delete(x3,not_picks,axis=1))
                    else:
                        tmp_x2.append(x3)
                    sum_val += x3.shape[0]
                    cnt += 1
            
            tmp_x1.append(tmp_x2)
        
        new_x.append(tmp_x1)
        new_y.append(np.array(tmp_y1))
    
    down_size = int((sum_val / cnt) * 160 / 250) 
    
    x = []
    
    for i in range(len(new_x)):
        tmp = []
        for j in range(len(new_x[i])):
            for k in range(len(new_x[i][j])):
                tmp.append(resample(new_x[i][j][k],down_size,axis=0))
        x.append(np.array(tmp))
    
    return x, new_y

In [217]:
x1t, y1t = down_data2(data_1t[:,0],data_1t[:,1],1)
x1e, y1e = down_data2(data_1e[:,0],data_1e[:,1],1)

C:\Anaconda3\lib\site-packages\scipy\signal\signaltools.py:2223: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
C:\Anaconda3\lib\site-packages\scipy\signal\signaltools.py:2225: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
C:\Anaconda3\lib\site-packages\scipy\signal\signaltools.py:2230: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result

In [218]:
def split_rest_mi(x):
    x_rest, x_mi = [], []
    for i, x1 in enumerate(x):
        tmp1, tmp2 = [], []
        for j, x2 in enumerate(x1):
            tmp1.append(x2[240:720,:])
            tmp2.append(x2[480:960,:])
        x_rest.append(np.array(tmp1))
        x_mi.append(np.array(tmp2))
        
    return np.array(x_rest), np.array(x_mi)

In [219]:
x1t_rest, x1t_mi = split_rest_mi(x1t)
x1e_rest, x1e_mi = split_rest_mi(x1e)

In [220]:
def write_npz_file(x,y,folder,head,subject):
    if not os.path.exists(folder):
        os.makedirs(folder)
    filename = str(folder)+"/S"+str(subject).zfill(2)+head+".npz"
    np.savez(filename,x=x,y=y)

In [156]:
def change_y_to_text(in_y):
    y = []
    for i in range(len(in_y)):
        if in_y[i] == 1:
            y.append('left')
        elif in_y[i] == 2:
            y.append('right')
        else:
            y.append('')
    
    return np.array(y)

In [221]:
y1t = [change_y_to_text(y) for y in y1t]
y1e = [change_y_to_text(y) for y in y1e]

In [258]:
"""
for i in range(len(x1t_rest)):
    write_npz_file(x1t_rest[i],y1t[i],"eeg_dataset1","t_rest",i+1)
    write_npz_file(x1t_mi[i],y1t[i],"eeg_dataset1","t_mi",i+1)

for i in range(len(x1e_rest)):
    write_npz_file(x1e_rest[i],y1e[i],"eeg_dataset1","e_rest",i+1)
    write_npz_file(x1e_mi[i],y1e[i],"eeg_dataset1","e_mi",i+1)
"""

'\nfor i in range(len(x1t_rest)):\n    write_npz_file(x1t_rest[i],y1t[i],"eeg_dataset1","t_rest",i+1)\n    write_npz_file(x1t_mi[i],y1t[i],"eeg_dataset1","t_mi",i+1)\n\nfor i in range(len(x1e_rest)):\n    write_npz_file(x1e_rest[i],y1e[i],"eeg_dataset1","e_rest",i+1)\n    write_npz_file(x1e_mi[i],y1e[i],"eeg_dataset1","e_mi",i+1)\n'

In [259]:
data_4t = np.array([extract_data(data) for data in raw_data4t])
data_4e = np.array([extract_data(data) for data in raw_data4e])

In [260]:
x4t, y4t = down_data2(data_4t[:,0],data_4t[:,1],4)
x4e, y4e = down_data2(data_4e[:,0],data_4e[:,1],4)

C:\Anaconda3\lib\site-packages\scipy\signal\signaltools.py:2223: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
C:\Anaconda3\lib\site-packages\scipy\signal\signaltools.py:2225: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
C:\Anaconda3\lib\site-packages\scipy\signal\signaltools.py:2233: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result

In [261]:
x4t = [ xx[:,:,:3] for xx in x4t]
x4e = [ xx[:,:,:3] for xx in x4e]

In [262]:
x4t_rest, x4t_mi = split_rest_mi(x4t)
x4e_rest, x4e_mi = split_rest_mi(x4e)

In [263]:
y4t = [change_y_to_text(y) for y in y4t]
y4e = [change_y_to_text(y) for y in y4e]

In [264]:
for i in range(len(x4t_rest)):
    write_npz_file(x4t_rest[i],y4t[i],"eeg_dataset4","t_rest",i+1)
    write_npz_file(x4t_mi[i],y4t[i],"eeg_dataset4","t_mi",i+1)

for i in range(len(x4e_rest)):
    write_npz_file(x4e_rest[i],y4e[i],"eeg_dataset4","e_rest",i+1)
    write_npz_file(x4e_mi[i],y4e[i],"eeg_dataset4","e_mi",i+1)